# Sale Prediction

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [50]:
# load data
categories = pd.read_csv("item_categories.csv")
items = pd.read_csv("items.csv")
shops = pd.read_csv("shops.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("sales_train.csv")

From the competition, it wants the final result to be item id and the item count per month. The prediction primarily depends on item_price item_id and item_cnt_day. Firstly, we need to clean the data.

In [51]:
train = train.drop(columns=['date'])

In [52]:
train.head(10)

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,59,22154,999.00,1.0
1,0,25,2552,899.00,1.0
2,0,25,2552,899.00,-1.0
3,0,25,2554,1709.05,1.0
4,0,25,2555,1099.00,1.0
5,0,25,2564,349.00,1.0
6,0,25,2565,549.00,1.0
7,0,25,2572,239.00,1.0
8,0,25,2572,299.00,1.0
9,0,25,2573,299.00,3.0


Check if the dataset is clean:

In [53]:
train.isna().sum()

date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [54]:
train.dtypes

date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

Then group the data to get what we need to predict:

In [56]:
train_by_month = train[train['item_cnt_day']!=-1.0].groupby(['date_block_num','shop_id','item_id']).agg({'item_price':'mean','item_cnt_day':'sum'})
train_by_month.head(10)

item_price  item_cnt_day
date_block_num shop_id item_id                          
0              0       32            221.0           6.0
                       33            347.0           3.0
                       35            247.0           1.0
                       43            221.0           1.0
                       51            128.5           2.0
                       61            195.0           1.0
                       75             76.0           1.0
                       88             76.0           1.0
                       95            193.0           1.0
                       96             70.0           1.0